# final

In [1]:
pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rgrws3ni
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rgrws3ni
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=8caea93407e14b792007b8bf2cbc3c5aa4dc20155f430be14832f6d07786e2c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-sxj48t_k/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import clip
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# 🔹 Chemins des fichiers
DATA_PATH = "/kaggle/input/human-motion-description-hmd-motion-to-text"
MOTIONS_PATH = os.path.join(DATA_PATH, "motions")
TEXTS_PATH = os.path.join(DATA_PATH, "texts")
train_file = os.path.join(DATA_PATH, "train.txt")
test_file = os.path.join(DATA_PATH, "test.txt")

# 🔹 Chargement des IDs
print("🔄 Chargement des IDs...")
train_ids = [line.strip() for line in open(train_file).readlines()]
test_ids = [line.strip() for line in open(test_file).readlines()]
print("✅ Chargement des IDs terminé !")

# 🔹 Chargement du modèle CLIP
print("🔄 Chargement du modèle CLIP...")
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _ = clip.load("ViT-B/32", device=device)
clip_model.eval()
print("✅ CLIP chargé avec succès !")

# 🔹 Chargement du modèle GPT-2
print("🔄 Chargement du modèle GPT-2...")
gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token  # Ajout du token de padding
gpt_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
print("✅ GPT-2 chargé avec succès !")

# 🔹 Fonction pour générer les embeddings texte avec CLIP
def get_text_embedding(text):
    tokenized_text = clip.tokenize([text[:75]]).to(device)
    with torch.no_grad():
        text_embedding = clip_model.encode_text(tokenized_text)
        return text_embedding.float() / text_embedding.norm(dim=-1, keepdim=True)

# 🔹 Dataset Motion + Text Embeddings
class MotionTextDataset(Dataset):
    def __init__(self, motion_ids):
        self.motion_ids = motion_ids

    def __len__(self):
        return len(self.motion_ids)

    def __getitem__(self, idx):
        motion_id = self.motion_ids[idx]
        motion = np.load(os.path.join(MOTIONS_PATH, f"{motion_id}.npy"))
        motion_tensor = torch.tensor(motion.reshape(motion.shape[0], -1), dtype=torch.float32)
        
        text_file = os.path.join(TEXTS_PATH, f"{motion_id}.txt")
        text = np.random.choice(open(text_file).readlines()).strip() if os.path.exists(text_file) else "No description available"
        text_embedding = get_text_embedding(text).squeeze()
        
        return motion_tensor, text_embedding, text

# 🔹 DataLoader
train_loader = DataLoader(MotionTextDataset(train_ids), batch_size=16, shuffle=True)

# 🔹 Encodeur basé sur LSTM
class MotionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_dim):
        super(MotionEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, latent_dim)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        return self.fc(hidden)

# 🔹 Initialisation du modèle
encoder = MotionEncoder(input_size=66, hidden_size=128, latent_dim=512).to(device)
optimizer = optim.AdamW(encoder.parameters(), lr=1e-4)
criterion = nn.MSELoss()
num_epochs = 3

# 🔹 Entraînement de l'encodeur
print("🚀 Début de l'entraînement...")
for epoch in range(num_epochs):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for motion, text_embedding, _ in progress_bar:
        motion, text_embedding = motion.to(device), text_embedding.to(device)
        optimizer.zero_grad()
        z_motion = encoder(motion)
        loss = criterion(z_motion, text_embedding)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
print("✅ Entraînement terminé !")

# 🔹 Sauvegarde du modèle
torch.save(encoder.state_dict(), "motion_encoder.pth")
print("✅ Encodeur sauvegardé avec succès !")



🔄 Chargement des IDs...
✅ Chargement des IDs terminé !
🔄 Chargement du modèle CLIP...


100%|███████████████████████████████████████| 338M/338M [00:23<00:00, 15.1MiB/s]


✅ CLIP chargé avec succès !
🔄 Chargement du modèle GPT-2...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Epoch 1/3, Loss: 0.0006930335275102315


Epoch 2/3: 100%|██████████| 814/814 [02:15<00:00,  6.00it/s, loss=0.000378]


Epoch 2/3, Loss: 0.00037780532630415594


Epoch 3/3: 100%|██████████| 814/814 [02:16<00:00,  5.95it/s, loss=0.000362]

Epoch 3/3, Loss: 0.00036236785227233766
✅ Entraînement terminé !
✅ Encodeur sauvegardé avec succès !


In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer  # On utilisera le tokenizer de GPT-2 pour le vocabulaire

device = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------------------------------------------
# Chemins des fichiers
# ------------------------------------------------------------
DATA_PATH = "/kaggle/input/human-motion-description-hmd-motion-to-text"
MOTIONS_PATH = os.path.join(DATA_PATH, "motions")
TEXTS_PATH = os.path.join(DATA_PATH, "texts")
train_file = os.path.join(DATA_PATH, "train.txt")
test_file = os.path.join(DATA_PATH, "test.txt")

with open(train_file, "r") as f:
    train_ids = [line.strip() for line in f.readlines()]
with open(test_file, "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

# ------------------------------------------------------------
# Utilisation du tokenizer de GPT-2 (pour le vocabulaire)
# ------------------------------------------------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# On s'assure d'avoir un token de padding. Par défaut GPT-2 n'en possède pas.
tokenizer.pad_token = tokenizer.eos_token

# ------------------------------------------------------------
# Dataset pour le décodeur
# Pour chaque motion, on charge la motion (.npy) et le texte associé (ex: "000009.txt")
# On prend la première ligne du texte (avant un éventuel '#' si présent)
# On prépare les inputs pour le teacher forcing en décalant d'un token.
# ------------------------------------------------------------
class MotionTextDecoderDataset(Dataset):
    def __init__(self, motion_ids, motions_path, texts_path, tokenizer, max_length=64):
        self.motion_ids = motion_ids
        self.motions_path = motions_path
        self.texts_path = texts_path
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.motion_ids)
    
    def __getitem__(self, idx):
        motion_id = self.motion_ids[idx]
        # Charger la motion
        motion_path = os.path.join(self.motions_path, f"{motion_id}.npy")
        motion = np.load(motion_path)
        motion_tensor = torch.tensor(motion.reshape(motion.shape[0], -1), dtype=torch.float32)
        
        # Charger le texte associé
        text_path = os.path.join(self.texts_path, f"{motion_id}.txt")
        if os.path.exists(text_path):
            with open(text_path, encoding="utf-8") as f:
                line = f.readline().strip()
            # Si le texte contient plusieurs descriptions séparées par '#', on prend la première
            text = line.split('#')[0].strip()
        else:
            text = "No description available"
        
        # Tokenisation
        encoding = self.tokenizer(text, truncation=True, max_length=self.max_length,
                                  padding="max_length", return_tensors="pt")
        input_ids = encoding["input_ids"].squeeze(0)  # séquence de tokens (longueur max_length)
        # Pour le teacher forcing, on décalle : 
        # On utilise input_ids[:-1] comme entrée et input_ids[1:] comme cible.
        decoder_input_ids = input_ids[:-1]
        target_ids = input_ids[1:]
        
        return motion_tensor, decoder_input_ids, target_ids

# ------------------------------------------------------------
# Définition de l'encodeur (le même que vous avez entraîné)
# ------------------------------------------------------------
class MotionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_dim):
        super(MotionEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, latent_dim)
    
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden)

encoder = MotionEncoder(input_size=66, hidden_size=128, latent_dim=512).to(device)
encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))
encoder.eval()
# On gèle l'encodeur pour l'entraînement du décodeur
for param in encoder.parameters():
    param.requires_grad = False

# ------------------------------------------------------------
# Définition du décodeur LSTM
# ------------------------------------------------------------
class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, input_ids, hidden):
        # input_ids : (B, T)
        embeds = self.embedding(input_ids)  # (B, T, embed_dim)
        outputs, hidden = self.lstm(embeds, hidden)  # outputs : (B, T, hidden_dim)
        logits = self.fc(outputs)  # (B, T, vocab_size)
        return logits, hidden

# ------------------------------------------------------------
# Modèle complet : l'encodeur (gelé) + le décodeur LSTM
# On initialise l'état caché du décodeur à partir de l'embedding de motion.
# ------------------------------------------------------------
class MotionToTextDecoder(nn.Module):
    def __init__(self, encoder, decoder, latent_dim, hidden_dim):
        super(MotionToTextDecoder, self).__init__()
        self.encoder = encoder  # Gelé
        self.decoder = decoder
        # Projection de l'embedding latent vers l'état initial du LSTM (hidden state)
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
    
    def forward(self, motion, decoder_input_ids):
        batch_size = motion.size(0)
        latent = self.encoder(motion)  # (B, latent_dim)
        # Initialisation de l'état caché et de la cellule
        h0 = self.latent_to_hidden(latent).unsqueeze(0)  # (1, B, hidden_dim)
        c0 = torch.zeros_like(h0)  # (1, B, hidden_dim)
        hidden = (h0, c0)
        logits, _ = self.decoder(decoder_input_ids, hidden)  # (B, T, vocab_size)
        return logits

vocab_size = len(tokenizer)
embed_dim = 512
hidden_dim = 512
decoder = LSTMDecoder(vocab_size, embed_dim, hidden_dim, num_layers=1).to(device)
model_decoder = MotionToTextDecoder(encoder, decoder, latent_dim=512, hidden_dim=hidden_dim).to(device)

# ------------------------------------------------------------
# Création du dataset et DataLoader pour l'entraînement du décodeur
# ------------------------------------------------------------
train_dataset_decoder = MotionTextDecoderDataset(train_ids, MOTIONS_PATH, TEXTS_PATH, tokenizer, max_length=64)
train_loader_decoder = DataLoader(train_dataset_decoder, batch_size=16, shuffle=True)

# ------------------------------------------------------------
# Entraînement du décodeur
# ------------------------------------------------------------
optimizer_decoder = optim.AdamW(model_decoder.parameters(), lr=1e-4)
criterion_decoder = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
num_epochs_decoder = 60

print("🚀 Début de l'entraînement du décodeur...")
model_decoder.train()
for epoch in range(num_epochs_decoder):
    total_loss = 0
    pbar = tqdm(train_loader_decoder, desc=f"Epoch {epoch+1}/{num_epochs_decoder}")
    for motion, decoder_input_ids, target_ids in pbar:
        motion = motion.to(device)
        decoder_input_ids = decoder_input_ids.to(device)
        target_ids = target_ids.to(device)
        
        optimizer_decoder.zero_grad()
        logits = model_decoder(motion, decoder_input_ids)  # (B, T, vocab_size)
        loss = criterion_decoder(logits.view(-1, vocab_size), target_ids.view(-1))
        loss.backward()
        optimizer_decoder.step()
        
        total_loss += loss.item()
        pbar.set_postfix(loss=f"{total_loss/(pbar.n+1):.4f}")
    print(f"Epoch {epoch+1}/{num_epochs_decoder}, Loss: {total_loss/len(train_loader_decoder):.4f}")
print("✅ Fin de l'entraînement du décodeur!")

# ------------------------------------------------------------
# Fonction d'inférence pour générer une phrase à partir d'une motion donnée
# ------------------------------------------------------------
def generate_sentence_decoder(model, motion_id, max_len=128):
    model.eval()
    # Charger la motion
    motion = np.load(os.path.join(MOTIONS_PATH, f"{motion_id}.npy"))
    motion_tensor = torch.tensor(motion.reshape(motion.shape[0], -1), dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        latent = model.encoder(motion_tensor)  # (1, 512)
        h0 = model.latent_to_hidden(latent).unsqueeze(0)  # (1, 1, hidden_dim)
        c0 = torch.zeros_like(h0)
        hidden = (h0, c0)
    
    # Initialiser avec un token de départ.
    # Si le tokenizer dispose d'un token BOS, on l'utilise, sinon on utilise EOS.
    start_token = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.eos_token_id
    input_token = torch.tensor([[start_token]], device=device)
    generated_tokens = [start_token]
    
    # Génération token par token
    for _ in range(max_len):
        with torch.no_grad():
            embeds = model.decoder.embedding(input_token)  # (1, 1, embed_dim)
            output, hidden = model.decoder.lstm(embeds, hidden)  # (1, 1, hidden_dim)
            logits = model.decoder.fc(output.squeeze(1))  # (1, vocab_size)
            next_token = torch.argmax(logits, dim=-1).unsqueeze(0)  # (1, 1) - Greedy search
            token_id = next_token.item()
            if token_id == tokenizer.eos_token_id:
                break
            generated_tokens.append(token_id)
            input_token = next_token
    sentence = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return sentence

# ------------------------------------------------------------
# Exemple d'inférence sur une motion du test set
# ------------------------------------------------------------
motion_id_example = test_ids[0]  # Par exemple, "000009"
sentence_generated = generate_sentence_decoder(model_decoder, motion_id_example, max_len=30)
print(f"Motion {motion_id_example} -> {sentence_generated}")


<ipython-input-3-803f2ea2f092>:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))


🚀 Début de l'entraînement du décodeur...


Epoch 1/60: 100%|██████████| 814/814 [01:01<00:00, 13.14it/s, loss=5.2163]


Epoch 1/60, Loss: 5.2163


Epoch 2/60: 100%|██████████| 814/814 [01:00<00:00, 13.45it/s, loss=3.9945]


...

Epoch 58/60, Loss: 0.7069


Epoch 59/60: 100%|██████████| 814/814 [00:59<00:00, 13.67it/s, loss=0.6960]


Epoch 59/60, Loss: 0.6951


Epoch 60/60: 100%|██████████| 814/814 [00:59<00:00, 13.77it/s, loss=0.6860]


Epoch 60/60, Loss: 0.6852
✅ Fin de l'entraînement du décodeur!
Motion 014295 ->  on the leg and start moving. they bring hands up and down while holding the legs apart. they then stand back up and down facing forward. 


In [4]:
# 🔹 Chargement du modèle sauvegardé (optionnel, si vous partez d'une nouvelle session)
encoder = MotionEncoder(input_size=66, hidden_size=128, latent_dim=512).to(device)
encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))
encoder.eval()  # Mode évaluation

# 🔹 Préparation du DataLoader pour l'ensemble de test
test_dataset = MotionTextDataset(test_ids)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 🔹 Boucle de test
total_test_loss = 0.0
criterion = nn.MSELoss()

with torch.no_grad():
    for motion, text_embedding, text in tqdm(test_loader, desc="Test"):
        motion, text_embedding = motion.to(device), text_embedding.to(device)
        predicted_embedding = encoder(motion)
        loss = criterion(predicted_embedding, text_embedding)
        total_test_loss += loss.item()

avg_test_loss = total_test_loss / len(test_loader)
print(f"Test Loss moyenne : {avg_test_loss:.4f}")


<ipython-input-4-ef8b41dfd7c8>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))
Test: 100%|███

Test Loss moyenne : 0.0005


In [21]:
import pandas as pd

# Mettez le modèle en mode évaluation
model_decoder.eval()

results = []
# Parcourir toutes les motions du test
for motion_id in test_ids:
    sentence = generate_sentence_decoder(model_decoder, motion_id, max_len=10)
    results.append({"id": motion_id, "text": sentence})
     # Optionnel : affiche chaque résultat

# Création du DataFrame et sauvegarde en CSV
submission = pd.DataFrame(results)
submission.to_csv("submissionv3.csv", index=False)


In [25]:
pd.read_csv("submission.csv")['text'].unique()

array([' on the leg and start moving. they bring hands up and down while holding the legs apart. they then stand back up and down facing forward.  he walks in a complete position before taking a circle back in the way he started to walk back a few times. he turns to the left and walks',
       ' is on the floor and crossing their legs. they are doing the handrail to walk back and forth. then the figure is walking back and forth. the figure turns and turns to the front. then they step back the opposite end of the plane. they are doing the air and turns and takes 3',
       ' the hands and band the legs to the legs. the person is walking and then backwards to the side. then they stop running, stop hands and put something down from behind them and then to stop. he looks like someone else is in fighting but and then they do running and then running from back to',
       ' is walking on a treadmill with arms outstretched. they step over the left foot while moving both arms up and down. he t

# Result : 0.2
# $$$

In [28]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

################################################################################
# 1) Paramètres & chemins
################################################################################
DATA_PATH = "/kaggle/input/human-motion-description-hmd-motion-to-text"
MOTIONS_PATH = os.path.join(DATA_PATH, "motions")
TEXTS_PATH   = os.path.join(DATA_PATH, "texts")

train_file = os.path.join(DATA_PATH, "train.txt")
val_file   = os.path.join(DATA_PATH, "val.txt")   # suppose qu'il existe
test_file  = os.path.join(DATA_PATH, "test.txt")

num_epochs         = 40
batch_size         = 16
embed_dim          = 512
hidden_dim         = 512
latent_dim         = 512
dropout_p          = 0.2
weight_decay_val   = 1e-2
learning_rate      = 1e-4
num_layers_decoder = 2  # => On aura besoin (2, B, hidden_dim) pour (h0, c0)

# Early stopping
patience       = 3
best_val_loss  = float('inf')
no_improve_cnt = 0

max_length_dataset = 128  # on a choisi 128 tokens

################################################################################
# 2) Chargement des IDs
################################################################################
with open(train_file, "r") as f:
    train_ids = [line.strip() for line in f]

with open(val_file, "r") as f:
    val_ids = [line.strip() for line in f]

with open(test_file, "r") as f:
    test_ids = [line.strip() for line in f]

################################################################################
# 3) Tokenizer GPT-2
################################################################################
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  
vocab_size = len(tokenizer)

################################################################################
# 4) Dataset
################################################################################
class MotionTextDecoderDataset(Dataset):
    def __init__(self, motion_ids, motions_path, texts_path, tokenizer, max_length=64):
        self.motion_ids   = motion_ids
        self.motions_path = motions_path
        self.texts_path   = texts_path
        self.tokenizer    = tokenizer
        self.max_length   = max_length

    def __len__(self):
        return len(self.motion_ids)
    
    def __getitem__(self, idx):
        motion_id = self.motion_ids[idx]
        
        # Charger la motion
        motion_path = os.path.join(self.motions_path, f"{motion_id}.npy")
        motion      = np.load(motion_path)
        motion_tensor = torch.tensor(motion.reshape(motion.shape[0], -1), dtype=torch.float32)
        
        # Charger le texte
        text_path = os.path.join(self.texts_path, f"{motion_id}.txt")
        if os.path.exists(text_path):
            with open(text_path, encoding="utf-8") as f:
                line = f.readline().strip()
            text = line.split('#')[0].strip()
        else:
            text = "No description available"
        
        # Tokenisation
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].squeeze(0)

        # Décalage
        decoder_input_ids = input_ids[:-1]
        target_ids        = input_ids[1:]
        
        return motion_tensor, decoder_input_ids, target_ids

################################################################################
# 5) Encodeur gelé
################################################################################
class MotionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_dim):
        super(MotionEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc   = nn.Linear(hidden_size * 2, latent_dim)
    
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden_cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden_cat)

encoder = MotionEncoder(input_size=66, hidden_size=128, latent_dim=latent_dim).to(device)
encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))
encoder.eval()
for param in encoder.parameters():
    param.requires_grad = False

################################################################################
# 6) Décodeur LSTM (2 couches + dropout=0.2)
################################################################################
class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, dropout=0.2, num_layers=2):
        super(LSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # Le dropout sera effectif entre les 2 couches
        self.lstm      = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, 
                                 batch_first=True, dropout=dropout)
        self.fc        = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, input_ids, hidden):
        embeds = self.embedding(input_ids)  # (B, T, embed_dim)
        outputs, hidden = self.lstm(embeds, hidden)  # outputs: (B, T, hidden_dim)
        logits = self.fc(outputs)                   # (B, T, vocab_size)
        return logits, hidden

################################################################################
# 7) Modèle global : encodeur gelé + décodeur
################################################################################
class MotionToTextDecoder(nn.Module):
    def __init__(self, encoder, decoder, latent_dim, hidden_dim, num_layers=2):
        super(MotionToTextDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.num_layers = num_layers  # pour répéter h0, c0

    def forward(self, motion, dec_in):
        latent = self.encoder(motion)      # (B, latent_dim)

        # Au lieu de (1, B, hidden_dim), on crée (num_layers, B, hidden_dim)
        # Si num_layers=2, on répète 2 fois la même init (ou on pourrait init différemment).
        h0_1 = self.latent_to_hidden(latent)              # (B, hidden_dim)
        h0   = h0_1.unsqueeze(0).repeat(self.num_layers, 1, 1) # (num_layers, B, hidden_dim)
        c0   = torch.zeros_like(h0)

        logits, _ = self.decoder(dec_in, (h0, c0))
        return logits

################################################################################
# 8) DataLoader (train + val)
################################################################################
train_dataset = MotionTextDecoderDataset(train_ids, MOTIONS_PATH, TEXTS_PATH, tokenizer, max_length_dataset)
val_dataset   = MotionTextDecoderDataset(val_ids,   MOTIONS_PATH, TEXTS_PATH, tokenizer, max_length_dataset)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

################################################################################
# 9) Instanciation modèle, optimizer & loss
################################################################################
decoder_model = LSTMDecoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_dim=hidden_dim,
    dropout=dropout_p,
    num_layers=num_layers_decoder
).to(device)

model_decoder = MotionToTextDecoder(
    encoder, 
    decoder_model, 
    latent_dim, 
    hidden_dim,
    num_layers=num_layers_decoder
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.AdamW(model_decoder.parameters(), lr=learning_rate, weight_decay=weight_decay_val)

################################################################################
# 10) Entraînement + Validation + Early Stopping
################################################################################
print("🚀 Début de l'entraînement (avec validation) ...")

for epoch in range(num_epochs):
    # --------------------
    # Phase d'entraînement
    # --------------------
    model_decoder.train()
    total_train_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
    for motion, dec_in, targets in pbar:
        motion  = motion.to(device)
        dec_in  = dec_in.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        logits = model_decoder(motion, dec_in)  # (B, T, vocab_size)
        loss   = criterion(logits.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        pbar.set_postfix({'loss': f"{total_train_loss/(pbar.n+1):.4f}"})
    
    avg_train_loss = total_train_loss / len(train_loader)
    
    # --------------------
    # Phase de validation
    # --------------------
    model_decoder.eval()
    total_val_loss = 0
    with torch.no_grad():
        for motion, dec_in, targets in val_loader:
            motion  = motion.to(device)
            dec_in  = dec_in.to(device)
            targets = targets.to(device)

            logits = model_decoder(motion, dec_in)
            val_loss = criterion(logits.view(-1, vocab_size), targets.view(-1))
            total_val_loss += val_loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Early stopping (simple exemple)
    global best_val_loss, no_improve_cnt
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_cnt = 0
        torch.save(model_decoder.state_dict(), "best_decoder.pth")
    else:
        no_improve_cnt += 1
        if no_improve_cnt >= patience:
            print(f"Early stopping déclenché (val loss stagne depuis {patience} époques).")
            break

print("✅ Fin de l'entraînement !")

################################################################################
# 11) Inférence (optionnel)
################################################################################
def generate_sentence_decoder(model, motion_id, max_len=50):
    model.eval()
    
    motion_path = os.path.join(MOTIONS_PATH, f"{motion_id}.npy")
    motion = np.load(motion_path)
    motion_tensor = torch.tensor(
        motion.reshape(motion.shape[0], -1), 
        dtype=torch.float32
    ).unsqueeze(0).to(device)

    with torch.no_grad():
        latent = model.encoder(motion_tensor)
        # h0 pour num_layers=2
        h0_1 = model.latent_to_hidden(latent)                # (1, hidden_dim)
        h0   = h0_1.unsqueeze(0).repeat(model.num_layers, 1, 1)  # (num_layers, 1, hidden_dim)
        c0   = torch.zeros_like(h0)
        hidden = (h0, c0)

    start_token = tokenizer.bos_token_id if tokenizer.bos_token_id else tokenizer.eos_token_id
    input_token = torch.tensor([[start_token]], device=device)
    generated_tokens = [start_token]

    for _ in range(max_len):
        with torch.no_grad():
            embeds = model.decoder.embedding(input_token)
            output, hidden = model.decoder.lstm(embeds, hidden)
            logits = model.decoder.fc(output.squeeze(1))

            next_token_id = torch.argmax(logits, dim=-1).unsqueeze(0)
            token_id = next_token_id.item()

            if token_id == tokenizer.eos_token_id:
                break

            generated_tokens.append(token_id)
            input_token = next_token_id
    
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text

# Exemple d'inférence (si tu veux tester sur un ID du val set)
if len(val_ids) > 0:
    sample_id = val_ids[0]
    gen_text = generate_sentence_decoder(model_decoder, sample_id, max_len=50)
    print(f"\nExemple génération (motion {sample_id}):\n{gen_text}")


<ipython-input-28-4a68db5ec712>:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))


🚀 Début de l'entraînement (avec validation) ...


Epoch 1/40 [Train]: 100%|██████████| 814/814 [01:38<00:00,  8.29it/s, loss=5.5372]


Epoch 1/40 | Train Loss: 5.5372 | Val Loss: 4.9799


Epoch 2/40 [Train]: 100%|██████████| 814/814 [01:38<00:00,  8.26it/s, loss=4.6884]


...

Epoch 38/40 | Train Loss: 1.1427 | Val Loss: 1.9391


Epoch 39/40 [Train]: 100%|██████████| 814/814 [01:35<00:00,  8.54it/s, loss=1.1089]


Epoch 39/40 | Train Loss: 1.1089 | Val Loss: 1.9172


Epoch 40/40 [Train]: 100%|██████████| 814/814 [01:35<00:00,  8.50it/s, loss=1.0817]


Epoch 40/40 | Train Loss: 1.0817 | Val Loss: 1.9017
✅ Fin de l'entraînement !

Exemple génération (motion M006646):
 standing on one leg, the person walks forward, turns left and walks to the right. they then stop facing backwards.  he then steps back to original position. he then stands back up by continuing to walk.  he steps forward and stops once


In [38]:
import pandas as pd

# Mettez le modèle en mode évaluation
model_decoder.eval()

results = []
# Parcourir toutes les motions du test
for motion_id in test_ids:
    sentence = generate_sentence_decoder(model_decoder, motion_id, max_len=50)
    results.append({"id": motion_id, "text": sentence})
     # Optionnel : affiche chaque résultat

# Création du DataFrame et sauvegarde en CSV
submission = pd.DataFrame(results)
submission.to_csv("submissionv4.csv", index=False)


In [39]:
pd.read_csv("submissionv4.csv")['text'].unique()

array([' person is doing a dance and stretches.  lifting both arms overhead, he tiptoes in a circle and returning to the spot with his hands. he stops, then returns to his original position. he then takes a few steps forward. he sits',
       ' standing, a person lifts his right hand to his face. he then lifts his hand and makes a small gesture with his right hand. he then raises his left hand, returns the object at his shoulder, and finally repeats a few times. then repeats',
       ' standing, a person steps forward, lifts their left hand up to mouth, then lowers it down and then walks backwards to the original position. they then raise their left hand and pull an object from waist height, placing the item then is swimming for a',
       ' person is walking forward slowly, bracing their hands together, and then their hands are together. they stop at the end of the push, and do them again. the person is using both hands together to see, and finally is in another of them',
       ' per

In [ ]:
#

# t

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"

################################################################################
# 1) Paramètres & chemins
################################################################################
DATA_PATH = "/kaggle/input/human-motion-description-hmd-motion-to-text"
MOTIONS_PATH = os.path.join(DATA_PATH, "motions")
TEXTS_PATH   = os.path.join(DATA_PATH, "texts")

train_file = os.path.join(DATA_PATH, "train.txt")
val_file   = os.path.join(DATA_PATH, "val.txt")
test_file  = os.path.join(DATA_PATH, "test.txt")

num_epochs         = 20   # Moins d'époques pour gagner du temps
batch_size         = 16
embed_dim          = 512
hidden_dim         = 512
latent_dim         = 512
dropout_p          = 0.2
weight_decay_val   = 1e-2
learning_rate      = 1e-4
num_layers_decoder = 2

# Early stopping
patience       = 3
best_val_loss  = float('inf')
no_improve_cnt = 0

max_length_dataset = 128  # Longueur max tokens

################################################################################
# 2) Chargement des IDs
################################################################################
with open(train_file, "r") as f:
    train_ids = [line.strip() for line in f]
with open(val_file, "r") as f:
    val_ids = [line.strip() for line in f]
with open(test_file, "r") as f:
    test_ids = [line.strip() for line in f]

################################################################################
# 3) Tokenizer GPT-2
################################################################################
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  
vocab_size = len(tokenizer)

################################################################################
# 4) Dataset (choix aléatoire de la description)
################################################################################
class MotionTextDecoderDataset(Dataset):
    def __init__(self, motion_ids, motions_path, texts_path, tokenizer, max_length=64):
        self.motion_ids   = motion_ids
        self.motions_path = motions_path
        self.texts_path   = texts_path
        self.tokenizer    = tokenizer
        self.max_length   = max_length

    def __len__(self):
        return len(self.motion_ids)
    
    def __getitem__(self, idx):
        motion_id = self.motion_ids[idx]
        
        # Charger la motion
        motion_path = os.path.join(self.motions_path, f"{motion_id}.npy")
        motion = np.load(motion_path)
        motion_tensor = torch.tensor(motion.reshape(motion.shape[0], -1), dtype=torch.float32)
        
        # Charger les lignes du fichier texte
        text_path = os.path.join(self.texts_path, f"{motion_id}.txt")
        if os.path.exists(text_path):
            with open(text_path, encoding="utf-8") as f:
                lines = [l.strip() for l in f.readlines() if l.strip()]
            # Choisir une ligne au hasard
            line = random.choice(lines)
            # Couper au '#' si besoin
            text = line.split('#')[0].strip()
        else:
            text = "No description available"
        
        # Tokenisation
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding="max_length",
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].squeeze(0)

        # Décalage
        decoder_input_ids = input_ids[:-1]
        target_ids        = input_ids[1:]
        
        return motion_tensor, decoder_input_ids, target_ids

################################################################################
# 5) Encodeur gelé
################################################################################
class MotionEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_dim):
        super(MotionEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc   = nn.Linear(hidden_size * 2, latent_dim)
    
    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden_cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden_cat)

encoder = MotionEncoder(input_size=66, hidden_size=128, latent_dim=latent_dim).to(device)
encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))
encoder.eval()
for param in encoder.parameters():
    param.requires_grad = False

################################################################################
# 6) Décodeur LSTM (2 couches + dropout)
################################################################################
class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, dropout=0.2, num_layers=2):
        super(LSTMDecoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm      = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, 
                                 batch_first=True, dropout=dropout)
        self.fc        = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, input_ids, hidden):
        embeds = self.embedding(input_ids)  
        outputs, hidden = self.lstm(embeds, hidden)
        logits = self.fc(outputs)
        return logits, hidden

################################################################################
# 7) Modèle complet
################################################################################
class MotionToTextDecoder(nn.Module):
    def __init__(self, encoder, decoder, latent_dim, hidden_dim, num_layers=2):
        super(MotionToTextDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.num_layers = num_layers

    def forward(self, motion, dec_in):
        latent = self.encoder(motion)  # (B, latent_dim)
        # h0 : (num_layers, B, hidden_dim)
        h0_1 = self.latent_to_hidden(latent)
        h0   = h0_1.unsqueeze(0).repeat(self.num_layers, 1, 1)
        c0   = torch.zeros_like(h0)
        logits, _ = self.decoder(dec_in, (h0, c0))
        return logits

################################################################################
# 8) DataLoader (train + val)
################################################################################
train_dataset = MotionTextDecoderDataset(train_ids, MOTIONS_PATH, TEXTS_PATH, tokenizer, max_length_dataset)
val_dataset   = MotionTextDecoderDataset(val_ids,   MOTIONS_PATH, TEXTS_PATH, tokenizer, max_length_dataset)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

################################################################################
# 9) Instanciation modèle, optimizer & loss
################################################################################
decoder_model = LSTMDecoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_dim=hidden_dim,
    dropout=dropout_p,
    num_layers=num_layers_decoder
).to(device)

model_decoder = MotionToTextDecoder(
    encoder, 
    decoder_model, 
    latent_dim, 
    hidden_dim,
    num_layers=num_layers_decoder
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.AdamW(model_decoder.parameters(), lr=learning_rate, weight_decay=weight_decay_val)

################################################################################
# 10) Entraînement + Validation + Early Stopping
################################################################################
print("🚀 Début de l'entraînement (avec validation) ...")

best_val_loss = float('inf')
no_improve_cnt = 0

for epoch in range(num_epochs):
    # --------------------
    # Phase d'entraînement
    # --------------------
    model_decoder.train()
    total_train_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
    for motion, dec_in, targets in pbar:
        motion  = motion.to(device)
        dec_in  = dec_in.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        logits = model_decoder(motion, dec_in)
        loss   = criterion(logits.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        pbar.set_postfix({'loss': f"{total_train_loss/(pbar.n+1):.4f}"})
    
    avg_train_loss = total_train_loss / len(train_loader)
    
    # --------------------
    # Phase de validation
    # --------------------
    model_decoder.eval()
    total_val_loss = 0
    with torch.no_grad():
        for motion, dec_in, targets in val_loader:
            motion  = motion.to(device)
            dec_in  = dec_in.to(device)
            targets = targets.to(device)

            logits = model_decoder(motion, dec_in)
            val_loss = criterion(logits.view(-1, vocab_size), targets.view(-1))
            total_val_loss += val_loss.item()
    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_cnt = 0
        torch.save(model_decoder.state_dict(), "best_decoder.pth")
    else:
        no_improve_cnt += 1
        if no_improve_cnt >= patience:
            print(f"Early stopping déclenché (val loss ne s'améliore plus depuis {patience} époques).")
            break

print("✅ Fin de l'entraînement !")

################################################################################
# 11) Inférence (optionnel)
################################################################################
def generate_sentence_decoder(model, motion_id, max_len=50):
    model.eval()
    
    motion_path = os.path.join(MOTIONS_PATH, f"{motion_id}.npy")
    motion = np.load(motion_path)
    motion_tensor = torch.tensor(motion.reshape(motion.shape[0], -1), 
                                 dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        latent = model.encoder(motion_tensor)
        h0_1 = model.latent_to_hidden(latent)  
        h0   = h0_1.unsqueeze(0).repeat(model.num_layers, 1, 1)
        c0   = torch.zeros_like(h0)
        hidden = (h0, c0)

    start_token = tokenizer.bos_token_id if tokenizer.bos_token_id else tokenizer.eos_token_id
    input_token = torch.tensor([[start_token]], device=device)
    generated_tokens = [start_token]

    for _ in range(max_len):
        with torch.no_grad():
            embeds = model.decoder.embedding(input_token)
            output, hidden = model.decoder.lstm(embeds, hidden)
            logits = model.decoder.fc(output.squeeze(1))

            next_token_id = torch.argmax(logits, dim=-1).unsqueeze(0)
            token_id = next_token_id.item()

            if token_id == tokenizer.eos_token_id:
                break

            generated_tokens.append(token_id)
            input_token = next_token_id
    
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text

# Exemple d'inférence
if len(val_ids) > 0:
    sample_id = val_ids[0]
    gen_text = generate_sentence_decoder(model_decoder, sample_id, max_len=50)
    print(f"\nExemple génération (motion {sample_id}):\n{gen_text}")


<ipython-input-40-b5711126feb2>:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load("motion_encoder.pth", map_location=device))


🚀 Début de l'entraînement (avec validation) ...


Epoch 1/20 [Train]: 100%|██████████| 814/814 [01:37<00:00,  8.31it/s, loss=5.5538]


Epoch 1/20 | Train Loss: 5.5538 | Val Loss: 5.0214


Epoch 2/20 [Train]: 100%|██████████| 814/814 [01:34<00:00,  8.60it/s, loss=4.7445]


...

Epoch 14/20 [Train]: 100%|██████████| 814/814 [01:41<00:00,  8.03it/s, loss=2.9052]


Epoch 14/20 | Train Loss: 2.9052 | Val Loss: 2.9634


Epoch 15/20 [Train]: 100%|██████████| 814/814 [01:40<00:00,  8.11it/s, loss=2.8492]


Epoch 15/20 | Train Loss: 2.8492 | Val Loss: 2.9426


Epoch 16/20 [Train]:  27%|██▋       | 220/814 [00:26<01:11,  8.25it/s, loss=2.7949]